In [55]:
!conda install -cf conda-forge pyogrio
#!pip install geopandas



187900.87s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



UnavailableInvalidChannel: HTTP 404 NOT FOUND for channel f <https://conda.anaconda.org/f>

The channel is not accessible or is invalid.

You will need to adjust your conda configuration to proceed.
Use `conda config --show channels` to view your configuration's current state,
and use `conda config --show-sources` to view config file locations.




In [50]:
!pip install owslib geopandas matplotlib ipyleaflet
from owslib.wfs import WebFeatureService
wfs = WebFeatureService(url='https://ows.emodnet-bathymetry.eu/wfs')
wfs

187594.52s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached ipyleaflet-0.19.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyogrio-0.9.0.tar.gz (352 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached branca-0.7.2-py3-none-any.whl.metadata (1.5 kB)
  Using cached jupyter_leaflet-0.19.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached traittypes-0.2.1-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached ipyleaflet-0.19.2-py3-none-any.whl (31 kB)
Using cached branca-0.7.2-py3-none-any.whl (25 kB)
Using cached jupyter_leaflet-0.19.2-py3-none-any.whl (1.1 MB)
Using cached traittypes-0.2.1-py2.py3-none-any.whl (8.6 kB)
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for pyogrio (pyproje

In [51]:
[operation.name for operation in wfs.operations]
wfs.version

'1.0.0'

In [17]:
list(wfs.contents)

['emodnet:download_tiles',
 'emodnet:contours',
 'emodnet:hr_bathymetry_area',
 'emodnet:quality_index',
 'world:sea_names',
 'emodnet:source_references',
 'gebco:undersea_features']

In [39]:
wfs.get_schema('emodnet:contours')
response = wfs.getfeature(typename='emodnet:contours', bbox=(54.521,7.084,58.692,15.196), srsname='urn:x-ogc:def:crs:EPSG:4326')

ServiceException: Feature type :e unknown

In [47]:

response = wfs.getfeature(typename="emodnet:contours", bbox=(54.521,7.084,58.692,15.196))

ServiceException: Feature type :e unknown

In [ ]:
out = open('~/Temp/quality_index.gml', 'wb')
out.write(bytes(response.read(), 'UTF-8'))
out.close()

In [22]:
!pip install geopandas
import geopandas as gpd
df = gpd.read_file("~/Temp/dk_coast_quality.geojson")

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyogrio-0.9.0.tar.gz (352 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for pyogrio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [151 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-cpython-310
      creating build/lib.macosx-10.9-x86_64-cpython-310/pyogrio
      copying pyogrio/_version.py -> build/lib.macosx-10.9-x86_64-cpython-310/pyogrio
      copying pyogrio/raw.py -> build/lib.macosx-10.9-x86_64-cpython-310/pyogrio
      copying pyogrio/util.py -> build/lib.macosx-10.9-x86_64-cpython-310/pyogrio
      copying pyogrio/__init__.py -> build/lib.macosx-10.9-x86_64-cpython-310/py

ModuleNotFoundError: No module named 'geopandas'

In [25]:
!pip install geoplot

import geoplot
import geoplot.crs as gcrs
geoplot.polyplot(
    df,
    projection=gcrs.AlbersEqualArea(),
    edgecolor='darkgrey',
    facecolor='lightgrey',
    linewidth=.3,
    figsize=(12, 8)
)

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyogrio-0.9.0.tar.gz (352 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyshp-2.3.1-py2.py3-none-any.whl.metadata (55 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 3.0 MB/s eta 0:00:0000:0100:01
Using cached pyshp-2.3.1-py2.py3-none-any.whl (46 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 2.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for pyogrio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [151 lines of output]
      running bdist_wheel
      running build
      running build_py
      creatin

ModuleNotFoundError: No module named 'geoplot'

In [53]:
from owslib.wfs import WebFeatureService
import geopandas as gpd
from shapely.geometry import shape
from ipyleaflet import Map, GeoData, basemaps

# Define the WFS service URL
wfs_url = "https://ows.emodnet-bathymetry.eu/wfs"

# Initialize the WFS client for version 1.0.0
wfs = WebFeatureService(url=wfs_url, version="1.0.0")

# List available feature types
feature_types = list(wfs.contents)
print("Available feature types:", feature_types)

# Define the layer (feature type) you want to query
layer = "your-layer-name-here"

# Define the BBOX covering Denmark (approximate coordinates: minx, miny, maxx, maxy)
bbox = (8.0, 54.5, 15.0, 57.8)

# Perform a GetFeature request with the BBOX filter
response = wfs.getfeature(
    typename=layer,
    bbox=bbox,
    outputFormat="json"  # You can specify the format (e.g., "json", "gml", etc.)
)

# Read the response data (assumes JSON output)
features = response.read()

# Convert the response to a GeoDataFrame
features_dict = json.loads(features)
features_gdf = gpd.GeoDataFrame.from_features(features_dict['features'])

# Check the coordinate reference system (CRS)
if features_gdf.crs is None:
    features_gdf.set_crs(epsg=4326, inplace=True)  # Assuming WGS 84 (EPSG:4326)

# Display the first few rows of the GeoDataFrame
print(features_gdf.head())

# Visualize the data on a map using ipyleaflet
center = [55.6761, 12.5683]  # Center map on Copenhagen, Denmark

m = Map(center=center, zoom=6, basemap=basemaps.OpenStreetMap.Mapnik)

geo_data = GeoData(geo_dataframe=features_gdf, style={'color': 'blue', 'opacity': 1, 'weight': 1.5},
                   hover_style={'color': 'red', 'fillOpacity': 0.7},
                   name="WFS Layer")

m.add_layer(geo_data)
m


ModuleNotFoundError: No module named 'geopandas'